In [11]:

%run ../../load_magic/storage.py
%run ../../load_magic/dataframes.py
import urllib.request
import re
from bs4 import BeautifulSoup
%pprint
%who

DATA_FOLDER: ../data/
SAVES_FOLDER: ../saves/
Pretty printing has been turned OFF
BeautifulSoup	 DATA_CSV_FOLDER	 DATA_FOLDER	 ENCODING_TYPE	 SAVES_CSV_FOLDER	 SAVES_FOLDER	 SAVES_PICKLE_FOLDER	 attempt_to_pickle	 column_descriptions_df	 
columns_list	 dataframes_dict	 example_iterrows	 get_column_descriptions	 get_link_html	 gunpowder_unit_list	 index_name	 load_csv	 load_dataframes	 
load_object	 mask_series	 math	 numeric_columns_list	 os	 pd	 pickle	 re	 save_dataframes	 
store_objects	 sys	 unit_availability_df	 urllib	 


In [2]:

index_name = 'Unit Name'
dataframes_dict = load_dataframes(unit_availability_df='unit_availability_df')
unit_availability_df = dataframes_dict['unit_availability_df']
if 'Unnamed: 0' in unit_availability_df.columns:
    unit_availability_df = unit_availability_df.set_index('Unnamed: 0')
elif index_name in unit_availability_df.columns:
    unit_availability_df = unit_availability_df.set_index(index_name)
unit_availability_df.index.name = index_name

Attempting to load C:\Users\dev\Documents\repositories\notebooks\age-of-empires-ii\saves\pickle\unit_availability_df.pickle.
No pickle exists - attempting to load C:\Users\dev\Documents\repositories\notebooks\age-of-empires-ii\saves\csv\unit_availability_df.csv.


In [81]:

# Turks: Gunpowder technologies are 50% cheaper.
mask_series = (unit_availability_df['Turks'] == 1)
mask_series = mask_series & ((unit_availability_df['In_Gunpowder_unit_Unit_Type'] == 1) |
                               (unit_availability_df['In_Gunpowder_unit_Armor_class'] == 1))
columns_list = ['Introduced in', 'Heading', 'Required research', 'Age']
unit_availability_df[mask_series][columns_list]

,Introduced in,Heading,Required research,Age
Unit Name,,,,
Hand Cannoneer,The Age of Kings,Archers,Chemistry,Imperial Age
Cannon Galleon,The Age of Kings,Navy,Cannon Galleon,Imperial Age
Elite Cannon Galleon,The Age of Kings,Navy,NaN,Imperial Age
Bombard Cannon,The Age of Kings,Siege,Chemistry,Imperial Age
Elite Janissary,The Age of Kings,Unique,NaN,Castle Age
Janissary,The Age of Kings,Unique,NaN,Castle Age


In [82]:

gunpowder_unit_list = unit_availability_df[mask_series].sort_index().index.tolist()
tech_list = ['Anarchy', 'Andean Sling', 'Architecture', 'Arquebus', 'Arrowslits', 'Arson',
             'Artillery', 'Atheism', 'Atlatl', 'Atonement', 'Ballistics', 'Banking', 'Bearded Axe',
             'Berserkergang', 'Blast Furnace', 'Block Printing', 'Bloodlines', 'Bodkin Arrow',
             'Boiling Oil', 'Bow Saw', 'Bracer', 'Caravan', 'Careening', 'Carrack', 'Cartography',
             'Castle Age', 'Chain Barding Armor', 'Chain Mail Armor', 'Chatras', 'Chemistry',
             'Chieftains', 'Chivalry', 'Coinage', 'Conscription', 'Couriers', 'Crenellations',
             'Crop Rotation', 'Double Crossbow', 'Double-Bit Axe', 'Drill', 'Druzhina', 'Dry Dock',
             'El Dorado', 'Faith', 'Farimba', 'Fervor', 'Feudal Age', 'Fletching', 'Forced Levy',
             'Forging', 'Furor Celtica', 'Garland Wars', 'Gillnets', 'Gold Mining',
             'Gold Shaft Mining', 'Great Wall', 'Greek Fire', 'Guilds', 'Hand Cart', 'Heated Shot',
             'Heavy Plow', 'Herbal Medicine', 'Heresy', 'Hoardings', 'Horse Collar', 'Howdah',
             'Husbandry', 'Illumination', 'Imperial Age', 'Inquisition', 'Iron Casting', 'Ironclad',
             'Kasbah', 'Kataparuto', 'Leather Archer Armor', 'Logistica', 'Loom', 'Madrasah',
             'Maghrabi Camels', 'Mahouts', 'Manipur Cavalry', 'Marauders', 'Masonry', 'Mercenaries',
             'Murder Holes', 'Nomads', 'Obsidian Arrows', 'Orthodoxy', 'Padded Archer Armor',
             'Panokseon', 'Paper Money', 'Parthian Tactics', 'Pavise', 'Perfusion',
             'Plate Barding Armor', 'Plate Mail Armor', 'Recurve Bow', 'Redemption',
             'Ring Archer Armor', 'Rocketry', 'Royal Heirs', 'Sanctity', 'Sappers',
             'Scale Barding Armor', 'Scale Mail Armor', 'Shatagni', 'Shinkichon', 'Shipwright',
             'Siege Engineers', 'Silk Road', 'Sipahi', 'Spies', 'Squires', 'Stone Mining',
             'Stone Shaft Mining', 'Stronghold', 'Sultans', 'Supremacy', 'Thalassocracy',
             'Theocracy', 'Thumb Ring', 'Tigui', 'Torsion Engines', 'Town Patrol', 'Town Watch',
             'Tracking', 'Treadmill Crane', 'Treason', 'Tusk Swords', 'Two-Man Saw', 'Warwolf',
             'Wheelbarrow', 'Yasama', 'Yeomen', 'Zealotry']
tech_regex = re.compile(r'\b({})\b'.format('|'.join(tech_list)))

In [83]:

def get_link_html(link):
    link_str = re.sub(' +', '_', link)
    link_url = 'https://ageofempires.fandom.com/wiki/{}'.format(link_str)
    try:
        extended_link_url = '{}_(Age_of_Empires_II)'.format(link_url)
        with urllib.request.urlopen(extended_link_url) as response:
            link_html = response.read()
        link_url = extended_link_url
    except:
        try:
            with urllib.request.urlopen(link_url) as response:
                link_html = response.read()
        except Exception as e:
            print('{} got this error: {}'.format(link_url, e))
            link_html = None
    
    return link_url, link_html

In [84]:

# Turks: Gunpowder technologies are 50% cheaper.
unit_child_list = []
unit_df_list = []
content_text_selector = '#mw-content-text'
for gunpowder_unit in gunpowder_unit_list:
    link_url, link_html = get_link_html(gunpowder_unit)
    if link_html is not None:
        summary_box_soup = BeautifulSoup(link_html, 'html.parser')
        content_text_div = summary_box_soup.select(content_text_selector)[0]
        children_list = list(content_text_div.children)
        for child in children_list:
            if child.name == 'table':
                if tech_regex.search(str(child)):
                    unit_child_list.append(child)
                    df = pd.read_html(str(child), header=0)[0]
                    unit_df_list.append(df)
len(unit_df_list)

https://ageofempires.fandom.com/wiki/Elite_Janissary got this error: HTTP Error 404: Not Found


5

In [93]:

# Turks: Gunpowder technologies are 50% cheaper.
tech_set = set()
for df in unit_df_list:
    df.columns = ['Key', 'Value']
    indexes_list = df.index.to_list()
    for idx in indexes_list[3:]:
        for match_obj in tech_regex.finditer(df.iloc[idx]['Value']):
            tech_set.add(match_obj.group())
mask_series = (technology_availability_df['Turks'] == 1)
mask_series = mask_series & (technology_availability_df.index.isin(tech_set))
tech_set = set(technology_availability_df[mask_series].index)
tech_set

{'Faith', 'Careening', 'Dry Dock', 'Conscription', 'Shipwright', 'Ring Archer Armor', 'Padded Archer Armor', 'Leather Archer Armor', 'Artillery', 'Heresy'}

In [71]:

# Turks: Gunpowder technologies are 50% cheaper.
tech_child_list = []
tech_df_list = []
content_text_selector = '#mw-content-text'
for gunpowder_tech in tech_set:
    link_url, link_html = get_link_html(gunpowder_tech)
    if link_html is not None:
        summary_box_soup = BeautifulSoup(link_html, 'html.parser')
        content_text_div = summary_box_soup.select(content_text_selector)[0]
        children_list = list(content_text_div.children)
        for child in children_list:
            if child.name == 'table':
                if tech_regex.search(str(child)):
                    tech_child_list.append(child)
                    df = pd.read_html(str(child), header=0)[0]
                    tech_df_list.append(df)
                    str_list = list(summary_box_soup.stripped_strings)
                    for i, html_str in enumerate(str_list):
                        if 'powder' in html_str.lower():
                            print()
                            print(gunpowder_tech)
                            print('str_list[{}]'.format(i))
                            print(html_str)
len(tech_df_list)


Arquebus
str_list[100]
Gunpowder units fire more accurately at moving targets

Arquebus
str_list[117]
gunpowder units

Arquebus
str_list[157]
Arquebus now has a smaller effect on gunpowder units, the speed modifier is reduced by 75%.

Arquebus
str_list[164]
Despite the reduced powder load, the arquebus was indeed more accurate than the hand cannons it replaced; this was primarily due to the fact that its matchlock mechanism allowed the weapon to be aimed with both hands and the attention of the gunner to be focused upon the target, not on priming their weapon.

Torsion Engines
str_list[176]
A Torsion Siege Engine is a type of artillery that utilizes torsion to launch projectiles. They were initially developed by the ancient Greeks, specifically Philip II of Macedon and Alexander the Great, and used through the Middle Ages until the development of gunpowder artillery in the 14th century proved them obsolete, essentially all non-battering ram/siege tower siege units utilize torsion. It 

17

In [72]:

index_name = 'Technology Name'
dataframes_dict = load_dataframes(technology_availability_df='technology_availability_df')
technology_availability_df = dataframes_dict['technology_availability_df']
if 'Unnamed: 0' in technology_availability_df.columns:
    technology_availability_df = technology_availability_df.set_index('Unnamed: 0')
elif index_name in technology_availability_df.columns:
    technology_availability_df = technology_availability_df.set_index(index_name)
technology_availability_df.index.name = index_name

Attempting to load C:\Users\dev\Documents\repositories\notebooks\age-of-empires-ii\saves\pickle\technology_availability_df.pickle.
No pickle exists - attempting to load C:\Users\dev\Documents\repositories\notebooks\age-of-empires-ii\saves\csv\technology_availability_df.csv.


In [87]:

column_descriptions_df = get_column_descriptions(technology_availability_df)
mask_series = column_descriptions_df['dtype'].isin(['int64', 'float64'])
numeric_columns_list = column_descriptions_df[mask_series]['column_name'].tolist()
print(column_descriptions_df.columns)
column_descriptions_df['dtype'].unique()

Index(['column_name', 'dtype', 'count_blanks', 'count_uniques', 'count_zeroes',
       'has_dates', 'min_value', 'max_value', 'only_integers'],
      dtype='object')


array(['int64', 'object'], dtype=object)

In [86]:

# Create the y variable
gunpowder_technologies_list = ['Chemistry', 'Arquebus', 'Shatagni']
def is_gunpowder_technology(row_series):
    if row_series.name in gunpowder_technologies_list:
        
        return True
    
    else:
        
        return False

y = technology_availability_df.apply(is_gunpowder_technology, axis=1).values

In [88]:

# Create the X variable
X = technology_availability_df[numeric_columns_list].fillna(0).values

In [89]:

from sklearn.ensemble import RandomForestClassifier

regr = RandomForestClassifier(max_depth=5, n_estimators=100)
regr.fit(X, y)
top_6_tuple_list = sorted([(cn, ci) for cn, ci in zip(numeric_columns_list,
                                                      regr.feature_importances_) if ci > 0.0],
       key=lambda x: x[1], reverse=True)[:6]

In [90]:

mask_series = (technology_availability_df.index.isin(gunpowder_technologies_list))
columns_list = [cn for (cn, cv) in top_6_tuple_list]
technology_availability_df[mask_series][columns_list].sort_values(columns_list, ascending=False)

,Food,Gold,Portuguese,Indians,Wood,Huns
Technology Name,,,,,,
Arquebus,700,400,1,0,0,0
Shatagni,500,300,0,1,0,0
Chemistry,300,200,1,1,0,1
